## Social Network Analysis

In [2]:
import tweepy
consumer_key = "YrlvTaYmfUFw6C3OnapAKiuaM"
consumer_secret = "QUZXmYYyyugyuT3GMNoOOxtTAst9fbWziGhcNsejDdsDU5tIbL"
access_token = "795922562546970624-6S2XzPe9K2gNrHphl3OUS1tjIgIh8rQ"
access_token_secret = "bfDz7r938dbfdfJai974vcQBeENz32VD8C4eBGZ71OECy"

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [4]:
api = tweepy.API(auth)

In [5]:
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print(tweet.text)

## Featching Twitter Data

In [6]:
user_name = "JohnCena"
users = api.lookup_users(screen_names=[user_name])
user = users[0]
type(user)

tweepy.models.User

In [7]:
tweets = user.timeline()
for i in range(10):
    tweet = tweets[i]
    print("Tweet (RT Count: {}) : \n\t{}\n".format(tweet.retweet_count , tweet.text))

Tweet (RT Count: 625) : 
	It's almost here...#BumblebeeMovie hits theaters THIS THURSDAY! 
Get your tickets at https://t.co/bgJa0LgSof! https://t.co/ptwjYiTFi8

Tweet (RT Count: 3363) : 
	Don’t rush to find out “what you want to be when you grow up”. But when you do find out, give that goal everything… https://t.co/mZng7oQ9zO

Tweet (RT Count: 2300) : 
	We are in a culture of speaking to get our point across. Actions, both immediate and long term, are the greatest de… https://t.co/LRFjSydVbK

Tweet (RT Count: 738) : 
	Tonight was for “us.” The privilege we have to entertain fans young and old has opened to the possibilities of what… https://t.co/kY0Cg7zt3G

Tweet (RT Count: 882) : 
	RT @bumblebeemovie: In TEN DAYS, Bumblebee arrives. Get tickets now to experience the movie that critics and fans can't stop buzzing about,…

Tweet (RT Count: 353) : 
	Thanks to everyone who attended last night @BNEventsGrove and to @BNBuzz for naming #ElbowGrease one of the best pi… https://t.co/RT08IcIiah

## Load Pretrained Model

In [8]:
import pickle
import numpy as np

In [9]:
loaded_model = pickle.load(open("detecting-fake-accounts/F_vs_G_model.data", "rb"))
model = loaded_model["classifier"]
scaler = loaded_model["scaler"]

In [10]:
useful_columns = ["statuses_count", "followers_count", "friends_count", "favourites_count", "listed_count", "default_profile", "profile_banner_url", "profile_background_tile", "profile_background_color" ,"verified"]

## Useful Columns
- statuses_count
- followers_count
- friends_count
- favourites_count
- listed_count
- default_profile
- profile_banner_url
- profile_background_tile
- profile_background_color
- verified

In [11]:
def parse_tweet(user_details):
    """ Parses user details for prediction """
    data = []
    for col in useful_columns:
        try:
                val = getattr(user_details, col)
                if isinstance(val, int):
                    data.append(val)
                elif isinstance(val, bool):
                    if val is True:
                        data.append(1)
                    else:
                        data.append(0)
                elif isinstance(val, str):
                    if col == 'profile_banner_url':
                        if val:
                            data.append(1)
                        else:
                            data.append(0)
                    elif col == 'profile_background_color':
                        if val != 'C0DEED':
                            data.append(1)
                        else:
                            data.append(0)
        except:
                data.append(0)
    return np.array(data)

In [12]:
for col in useful_columns:
    val = getattr(tweet.user, col)
    print("attribute : {}, value : {}".format(col, val))

attribute : statuses_count, value : 5931
attribute : followers_count, value : 11450831
attribute : friends_count, value : 2879
attribute : favourites_count, value : 7
attribute : listed_count, value : 33534
attribute : default_profile, value : False
attribute : profile_banner_url, value : https://pbs.twimg.com/profile_banners/141664648/1539882540
attribute : profile_background_tile, value : False
attribute : profile_background_color, value : 000000
attribute : verified, value : True


In [13]:
tweet.user.screen_name

'JohnCena'

In [14]:
parsed_tweet = parse_tweet(tweet.user)
print(*parsed_tweet, sep='\n')

5931
11450831
2879
7
33534
0
1
0
1
1


In [15]:
scaled_tweet = scaler.transform(parsed_tweet.reshape(1, -1))
model.predict_proba(scaled_tweet)

array([[0.71018517, 0.28981483]], dtype=float32)

In [16]:
class FakeNewsPredictor():
    
    def __init__(self, useful_columns, model_path):
        self._useful_columns = useful_columns
        self._model_path = model_path
        self._load_model()
        
    def _load_model(self):
        import pickle
        model = pickle.load(open(self._model_path, "rb"))
        self._classifier = model["classifier"]
        self._scaler = model["scaler"]
    
    def predict(self, users_details):
        fake_news_prob = 1
        fake_users = []
        for user in users_details:
            user = self._parse_details_v2(user)
            user = self._scaler.transform(user)
            probabilities = self._classifier.predict_proba(user)
            fake_news_prob *= probabilities[0][0]
            if np.argmax(probabilities[0]) == 0:
                fake_users.append(1)
            else:
                fake_users.append(0)
        return fake_news_prob ** (1/len(users_details)), fake_users
    
    def _parse_details_v1(self, details):
        """ Parses user details for prediction """
        data = []
        for column in self._useful_columns:
            try:
                value = getattr(details, column)
                if value == False:
                    data.append(0)
                elif value == True:
                    data.append(1)
                else:
                    data.append(int(value))
            except:
                # Detail not present
                data.append(0)
        return np.array(data).reshape(1, len(useful_columns))
    
    def _parse_details_v2(self, user_details):
        """ Parses user details for prediction """
        data = []
        for col in self._useful_columns:
            try:
                val = getattr(user_details, col)
                if isinstance(val, int):
                    data.append(val)
                elif isinstance(val, bool):
                    if val is True:
                        data.append(1)
                    else:
                        data.append(0)
                elif isinstance(val, str):
                    if col == 'profile_banner_url':
                        if val:
                            data.append(1)
                        else:
                            data.append(0)
                    elif col == 'profile_background_color':
                        if val != 'C0DEED':
                            data.append(1)
                        else:
                            data.append(0)
            except:
                data.append(0)
        return np.array(data).reshape(1, -1)

In [17]:
fakenewsmodel = FakeNewsPredictor(useful_columns, "detecting-fake-accounts/fake_account_model.dat")

### Inference

In [18]:
tweet = tweets[3]
tweet.text

'Tonight was for “us.” The privilege we have to entertain fans young and old has opened to the possibilities of what… https://t.co/kY0Cg7zt3G'

In [19]:
retweets = api.retweets(tweet.id, count=1000)

In [20]:
len(retweets)

87

In [94]:
users_details = [retweeter.user for retweeter in retweets]

In [95]:
for i, user in enumerate(users_details):
    print("{} : {}".format(i, user.screen_name))

0 : ManasBhattrai
1 : lettuceleagu3
2 : ImAddicted2WWE
3 : JohnCeena1295
4 : AustinAwesome7
5 : WWE_THECHAMP
6 : MsAartByHeart
7 : satcenation
8 : MelinaCenaFans
9 : JayCabriole
10 : AiganaeE
11 : Deanpaigegirl12
12 : TheGreatPickles
13 : AaronCena150
14 : BaKothiya
15 : johncen_wwe
16 : 1kamausamwel
17 : SeYvSR4m5VSV9Bl
18 : HarleyArvanitis
19 : ewa7497
20 : Yasharth0110163
21 : Abhishek13Dubey
22 : istathukku
23 : pinkteddy_bella
24 : MinHanVvV
25 : M_S_Mish
26 : Cbingham88
27 : roxana_carrero
28 : XXXCreepyDude
29 : PikaXRich
30 : bashora_89
31 : Signedx_
32 : Dekared1
33 : KGB65FLY
34 : Damian0122
35 : jasmicaelista
36 : dam8013
37 : zahidpro90
38 : JackHesselden
39 : gjbsmal
40 : Awadinho83
41 : HunchoRinyo
42 : jonathan_orbes
43 : DavidRayvonne
44 : NeftWWE
45 : theamanofficial
46 : SydneySK_
47 : GTSfan4lif
48 : NaomiFlynn14
49 : vmhelm2
50 : KaiseyF
51 : DpXTV
52 : Msami79979214
53 : finn_devitt
54 : tj_teddyrocks
55 : JuanG012284
56 : mody3189
57 : AkshayChauhan0
58 : kocev_fi

In [96]:
prediction = fakenewsmodel.predict(users_details)

In [97]:
prediction[0]

0.3188636335220946

In [98]:
print('Possible Fake Accounts')
for idx in range(len(users_details)):
    if prediction[1][idx]:
        print("{} : {}".format(idx, users_details[idx].screen_name))

Possible Fake Accounts
3 : JohnCeena1295
5 : WWE_THECHAMP


**User Name** : narendramodi

**Tweet**     : 'At 7:45 AM IST tomorrow, I will be delivering the keynote address at the Singapore Fintech Festival. I would be tal… https://t.co/S7oOFrMFBl'

**Possibility of Fake News** : ~0.0

**Possible Fake Accounts**   :  
- RaviSin41703280
- Samrat82037914
